In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/food5k/Food-5K/validation/0_307.jpg
/kaggle/input/food5k/Food-5K/validation/0_249.jpg
/kaggle/input/food5k/Food-5K/validation/0_217.jpg
/kaggle/input/food5k/Food-5K/validation/0_354.jpg
/kaggle/input/food5k/Food-5K/validation/0_287.jpg
/kaggle/input/food5k/Food-5K/validation/1_496.jpg
/kaggle/input/food5k/Food-5K/validation/1_453.jpg
/kaggle/input/food5k/Food-5K/validation/0_31.jpg
/kaggle/input/food5k/Food-5K/validation/1_163.jpg
/kaggle/input/food5k/Food-5K/validation/1_463.jpg
/kaggle/input/food5k/Food-5K/validation/0_16.jpg
/kaggle/input/food5k/Food-5K/validation/0_168.jpg
/kaggle/input/food5k/Food-5K/validation/0_419.jpg
/kaggle/input/food5k/Food-5K/validation/1_246.jpg
/kaggle/input/food5k/Food-5K/validation/0_228.jpg
/kaggle/input/food5k/Food-5K/validation/0_24.jpg
/kaggle/input/food5k/Food-5K/validation/1_8.jpg
/kaggle/input/food5k/Food-5K/validation/1_484.jpg
/kaggle/input/food5k/Food-5K/validation/0_362.jpg
/kaggle/input/food5k/Food-5K/validation/0_49.jpg
/kaggl

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
def prepare_df(data_type):
    X = []
    y = []
    path = '/kaggle/input/food5k/Food-5K/' + data_type + '/'

    for i in os.listdir(path):
        # Image
        X.append(i)
        # Label
        y.append(i.split('_')[0])

    X = np.array(X)
    y = np.array(y)

    df = pd.DataFrame()
    df['filename'] = X
    df['label'] = y

    return df


df_train = prepare_df('training')
df_val = prepare_df('validation')
df_test = prepare_df('evaluation')

In [4]:
df_train.head()

,filename,label
0,0_307.jpg,0
1,1_675.jpg,1
2,1_795.jpg,1
3,0_735.jpg,0
4,1_617.jpg,1


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create the ImageDataGenerator object
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
) 

# Generate batches and augment the images
train_generator = train_datagen.flow_from_dataframe(
    df_train,
    directory='/kaggle/input/food5k/Food-5K/training/',
    x_col='filename',
    y_col='label',
    class_mode='binary',
    target_size=(224, 224),
)

val_generator = train_datagen.flow_from_dataframe(
    df_val,
    directory='/kaggle/input/food5k/Food-5K/validation/',
    x_col='filename',
    y_col='label',
    class_mode='binary',
    target_size=(224, 224),
)

Found 3000 validated image filenames belonging to 2 classes.
Found 1000 validated image filenames belonging to 2 classes.


In [17]:
test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

test_generator = test_datagen.flow_from_dataframe(
    df_test,
    directory='/kaggle/input/food5k/Food-5K/evaluation/',
    x_col='filename',
    y_col='label',
    class_mode='binary',
    target_size=(224, 224),
)

Found 1000 validated image filenames belonging to 2 classes.


In [6]:
from tensorflow.keras.applications import ResNet50

# Initialize the Pretrained Model
feature_extractor = ResNet50(weights='imagenet', 
                             input_shape=(224, 224, 3),
                             include_top=False)

# Set this parameter to make sure it's not being trained
feature_extractor.trainable = False

# Set the input layer
input_ = tf.keras.Input(shape=(224, 224, 3))

# Set the feature extractor layer
x = feature_extractor(input_, training=False)

# Set the pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Set the final layer with sigmoid activation function
output_ = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Create the new model object
model = tf.keras.Model(input_, output_)

# Compile it
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

# Print The Summary of The Model
model.summary()

2021-12-29 12:51:36.888527: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-29 12:51:36.997166: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-29 12:51:36.997940: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-29 12:51:36.999224: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

94781440/94765736 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,589,761
Trainable params: 2,049
Non-trainable params: 23,587,712
_________________________________________________________________


In [7]:
model.fit(train_generator, epochs=20, validation_data=val_generator)

/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
2021-12-29 12:51:43.384433: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2021-12-29 12:51:47.664068: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


94/94 [==============================] - 81s 767ms/step - loss: 0.1971 - accuracy: 0.9253 - val_loss: 0.1086 - val_accuracy: 0.9620
Epoch 2/20
94/94 [==============================] - 60s 639ms/step - loss: 0.1002 - accuracy: 0.9660 - val_loss: 0.0887 - val_accuracy: 0.9690
Epoch 3/20
94/94 [==============================] - 61s 645ms/step - loss: 0.0817 - accuracy: 0.9717 - val_loss: 0.0851 - val_accuracy: 0.9640
Epoch 4/20
94/94 [==============================] - 60s 640ms/step - loss: 0.0667 - accuracy: 0.9770 - val_loss: 0.0814 - val_accuracy: 0.9680
Epoch 5/20
94/94 [==============================] - 60s 645ms/step - loss: 0.0590 - accuracy: 0.9793 - val_loss: 0.0658 - val_accuracy: 0.9760
Epoch 6/20
94/94 [==============================] - 61s 644ms/step - loss: 0.0559 - accuracy: 0.9827 - val_loss: 0.0831 - val_accuracy: 0.9670
Epoch 7/20
94/94 [==============================] - 60s 643ms/step - loss: 0.0480 - accuracy: 0.9860 - val_loss: 0.0598 - val_accuracy: 0.9770
Epoch 8/20

In [19]:
results = model.evaluate(test_generator)
print("test loss, test acc:", results)

32/32 [==============================] - 21s 641ms/step - loss: 0.0805 - accuracy: 0.9700
test loss, test acc: [0.08050109446048737, 0.9700000286102295]


In [20]:
prediction = model.predict(test_generator)

In [25]:
prediction.shape

(1000, 1)

In [30]:
import shutil

shutil.make_archive('resnet50_food_model', 'zip', './')

'/kaggle/working/resnet50_food_model.zip'